In [18]:
import math

def standard_normal_sf(x: float) -> float:
    """
    Standard normal survival function, SF(x) = 1 - Phi(x),
    implemented using math.erfc from the standard library.

    SF(x) = 0.5 * erfc(x / sqrt(2)).
    """
    return 0.5 * math.erfc(x / math.sqrt(2.0))

def standard_normal_sf_derivative(x: float) -> float:
    """
    Derivative of the standard normal survival function SF(x).
    This is -phi(x), where phi(x) is the standard normal PDF.
    """
    # phi(x) = 1/sqrt(2π) * exp(-x^2/2)
    pdf_x = math.exp(-0.5 * x**2) / math.sqrt(2.0 * math.pi)
    return -pdf_x

def _rational_approximation(p: float) -> float:
    """
    Classic Abramowitz & Stegun approximation (formula 26.2.23).
    """
    # Coefficients
    c = [2.515517, 0.802853, 0.010328]
    d = [1.432788, 0.189269, 0.001308]

    if p > 0.5:
        t = math.sqrt(-2.0 * math.log(1-p))
        f = -1.0
    else:
        t = math.sqrt(-2.0 * math.log(p))
        f = +1.0

    numerator = (c[2]*t + c[1])*t + c[0]
    denominator = ((d[2]*t + d[1])*t + d[0])*t + 1.0
    return f*(t - numerator / denominator)

def rational_approximation_py_native(p):
    low_p = float(p < 0.5)
    f = low_p * 2 - 1
    q = p*f + (1 - low_p) # robust for extreme low p
    t = math.sqrt(-2.0 * math.log(q))
    c0, c1, c2 = 2.515517, 0.802853, 0.010328
    d0, d1, d2 = 1.432788, 0.189269, 0.001308
    numerator = (c2 * t + c1) * t + c0
    denominator = ((d2 * t + d1) * t + d0) * t + 1.0
    return f*(t - numerator / denominator)

def standard_normal_isf_newton(p: float,
                               tol: float = 1e-10,
                               max_iter: int = 50,
                               verbose: int = 0) -> float:
    """
    Compute the ISF (inverse survival function) for the standard normal
    by solving SF(x) = p via Newton–Raphson starting from x0.

    Args:
        p       : Probability in (0,1).
        x0      : Initial guess for x.
        tol     : Convergence tolerance.
        max_iter: Maximum number of iterations.

    Returns:
        The value of x such that SF(x) ≈ p.
    """
    if not (0 < p < 1):
        raise ValueError("p must be in (0,1).")

    z = rational_approximation_py_native(p)
    count_iter = 0
    for _ in range(max_iter):
        # f(x)   = SF(x) - p
        # f'(x)  = SF'(x)
        f_val = standard_normal_sf(z) - p
        f_prime = standard_normal_sf_derivative(z)

        # Avoid dividing by zero (or extremely small derivative)
        if verbose > 1:
            print(f'{z=} {f_val=} {f_prime=} dx={- f_val / f_prime}')
        if abs(f_prime) * 1e14 <= abs(f_val):
            if verbose:
                print(f'{f_val=} {f_prime=}')
            break
        count_iter += 1
        dz = - f_val / f_prime
        z += dz
        if abs(dz) < tol:
            break # Converged
    if verbose:
        print(f'standard_normal_isf_newton {p=} {count_iter=}')
    return z  # Return last iterate if not converged within max_iter

######################################################

import numpy as np
from scipy.stats import norm

test_p_values = [2.0**(-65), 1e-20, 1e-15, 1e-12, 1e-9, 1e-6, 0.01, 0.05, 0.5, 0.95, 0.99]
for p in test_p_values:
    my_isf = standard_normal_isf_newton(p, verbose=1)
    scipy_isf = norm.isf(p)
    print(f"p = {p: .2f}, My ISF: {my_isf: .6f}, SciPy ISF: {scipy_isf: .6f}")

_ = standard_normal_isf_newton(p=1e-20, verbose=2)


standard_normal_isf_newton p=2.710505431213761e-20 count_iter=3
p =  0.00, My ISF:  9.155294, SciPy ISF:  9.155294
standard_normal_isf_newton p=1e-20 count_iter=3
p =  0.00, My ISF:  9.262340, SciPy ISF:  9.262340
standard_normal_isf_newton p=1e-15 count_iter=3
p =  0.00, My ISF:  7.941345, SciPy ISF:  7.941345
standard_normal_isf_newton p=1e-12 count_iter=3
p =  0.00, My ISF:  7.034484, SciPy ISF:  7.034484
standard_normal_isf_newton p=1e-09 count_iter=3
p =  0.00, My ISF:  5.997807, SciPy ISF:  5.997807
standard_normal_isf_newton p=1e-06 count_iter=3
p =  0.00, My ISF:  4.753424, SciPy ISF:  4.753424
standard_normal_isf_newton p=0.01 count_iter=3
p =  0.01, My ISF:  2.326348, SciPy ISF:  2.326348
standard_normal_isf_newton p=0.05 count_iter=3
p =  0.05, My ISF:  1.644854, SciPy ISF:  1.644854
standard_normal_isf_newton p=0.5 count_iter=2
p =  0.50, My ISF: -0.000000, SciPy ISF:  0.000000
standard_normal_isf_newton p=0.95 count_iter=3
p =  0.95, My ISF: -1.644854, SciPy ISF: -1.644854

In [3]:
import math
import numpy as np
from numba import njit, prange

@njit
def standard_normal_sf_derivative_njit(x: float) -> float:
    """
    Derivative of the standard normal survival function SF(x).
    This is -phi(x), where phi(x) is the standard normal PDF.
    """
    # phi(x) = 1/sqrt(2π) * exp(-x^2/2)
    pdf_x = math.exp(-0.5 * x**2) / math.sqrt(2.0 * math.pi)
    return -pdf_x

@njit
def standard_normal_sf_njit(x: float) -> float:
    """
    Standard normal survival function, SF(x) = 1 - Phi(x),
    implemented using math.erfc from the standard library.

    SF(x) = 0.5 * erfc(x / sqrt(2)).
    """
    return 0.5 * math.erfc(x / math.sqrt(2.0))


@njit(parallel=True)
def standard_normal_isf_njit(p: np.ndarray,
                               tol: float = 1e-10,
                               max_iter: int = 50) -> np.ndarray:
    """
    Compute the ISF (inverse survival function) for the standard normal
    by solving SF(x) = p via Newton–Raphson starting from x0.

    Args:
        p       : Probability in (0,1).
        tol     : Convergence tolerance.
        max_iter: Maximum number of iterations.

    Returns:
        The value of x such that SF(x) ≈ p.
    """
    if p.min() <= 0.0 or 1.0 <= p.max():
        raise ValueError("p must be in (0,1).")

    result = np.empty_like(p)
    for i in prange(p.size):
        pi = p[i]
        x: float = 0.0
        for _ in range(max_iter):
            # f(x)   = SF(x) - p
            # f'(x)  = SF'(x)
            f_val = standard_normal_sf_njit(x) - pi
            f_prime = standard_normal_sf_derivative_njit(x)

            # Avoid dividing by zero (or extremely small derivative)
            if abs(f_prime) * 1e14 <= abs(f_val):
                break
            dx = - f_val / f_prime
            x += dx
            if abs(dx) < tol:
                break # Converged
        result[i] = x
    return result 


In [4]:
import math
import numpy as np
from numba import cuda

@cuda.jit
def standard_normal_isf_kernel(p_array, out_array):
    """
    CUDA kernel to compute the standard normal ISF for each entry in p_array.
    Each thread processes exactly one element (threadIdx.x + blockIdx.x*blockDim.x).
    """
    i = cuda.grid(1) # type: ignore
    if i < p_array.size:
        p = p_array[i]
        # We can inline the approximation logic here or call a device function
        if p <= 0.0 or p >= 1.0:
            out_array[i] = math.nan
            return
        
        out_array[i] = _rational_approximation_device(p)


@cuda.jit(device=True)
def _rational_approximation_device(p):
    low_p = int(p < 0.5)
    q = 2*low_p * p + 1 - low_p - p
    f = low_p * 2 - 1
    t = math.sqrt(-2.0 * math.log(q))
    c0, c1, c2 = 2.515517, 0.802853, 0.010328
    d0, d1, d2 = 1.432788, 0.189269, 0.001308

    numerator = (c2 * t + c1) * t + c0
    denominator = ((d2 * t + d1) * t + d0) * t + 1.0
    return f*(t - numerator / denominator)


def standard_normal_isf_cuda(p_array_h):
    """
    Host function that copies p_array_h (NumPy on CPU) to the GPU,
    launches the kernel, and returns results to the CPU.
    """
    p_array_d = cuda.to_device(p_array_h)
    out_array_d = cuda.device_array_like(p_array_d)
    
    threads_per_block = 128
    blocks = (p_array_h.size + (threads_per_block - 1)) // threads_per_block
    
    standard_normal_isf_kernel[blocks, threads_per_block](p_array_d, out_array_d) # type: ignore
    
    return out_array_d.copy_to_host()


# Example usage
if __name__ == "__main__":
    p_values = np.array([1e-20, 1e-12, 1e-9, 1e-6, 0.01, 0.05, 0.5, 0.95, 0.99], dtype=np.float64)
    results = standard_normal_isf_cuda(p_values)
    print("p-values:", p_values)
    print("ISF CUDA results:", results)
    results = standard_normal_isf_njit(p_values)
    print("ISF NJIT results:", results)


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


p-values: [1.0e-20 1.0e-12 1.0e-09 1.0e-06 1.0e-02 5.0e-02 5.0e-01 9.5e-01 9.9e-01]
ISF CUDA results: [            nan  7.03405338e+00  5.99743792e+00  4.75325846e+00
  2.32678533e+00  1.64521144e+00  1.01006675e-07 -1.64521144e+00
 -2.32678533e+00]
ISF NJIT results: [ 9.26234009  7.03448383  5.99780702  4.75342431  2.32634787  1.64485363
  0.         -1.64485363 -2.32634787]
